# TorchSig: Impairments

This notebook describes TorchSig's concept of **impairments** and the parameterized Transform lists that comprise different TorchSig impairment levels.

---

## TorchSig Impairment Levels
The Impairments class is a special type of Transform that defines prepackaged lists of parameterized Transforms that are applied in a particular order to generate TorchSig datasets.

```python
from torchsig.transforms.impairments import Impairments
```

Impairments is usually employed within generation of TorchSig datasets where the specified **level** parameter guides which Transform list is used for data generation. The different levels were created by Radio Frequency engineers familiar with benchtop and over-the-air device testing. Currently, the provided impairments are\:
- Level 0: Perfect (digital) environment
- Level 1: Cabled (RF benchtop) environment
- Level 2: Wireless (RF over-the-air) environment

For levels greater than 0, the components of each Impairment capture the in-order TorchSig Transforms that model transmitter, channel, and receiver effects.

## Applying Impairments to datasets
Each Impairments instance holds two Transforms, representing the composition of all impairments to be applied to individual bursts within a signal, and the impairments to be applied to the signal as a whole.

These transforms, like any other transform or callable object, can be passed into Torchsig datasets.

In the example below, we are applying level 2 impairments (more detail on impairment levels below) to our dataset.

In [ ]:
from torchsig.utils.defaults import TorchSigDefaults
from torchsig.datasets.datasets import TorchSigIterableDataset
from torchsig.transforms.impairments import Impairments

In [ ]:
impairments = Impairments(level=2)
burst_impairments = impairments.signal_transforms
whole_signal_impairments = impairments.dataset_transforms

burst_impairments, whole_signal_impairments

In [ ]:
from pprint import pprint

dataset_metadata = TorchSigDefaults().default_dataset_metadata
dataset_metadata["num_iq_samples_dataset"] = 1024
dataset_metadata["fft_size"] = 64
dataset_metadata["num_signals_min"] = 1
dataset_metadata["num_signals_max"] = 3
pprint(dataset_metadata)

In [ ]:
pprint(dataset_metadata)
dataset = TorchSigIterableDataset(
    metadata=dataset_metadata,
    transforms=[whole_signal_impairments],
    component_transforms=[burst_impairments],
)

## Impairments: Machine Learning Transforms 
The Machine Learning (ML) Transforms are a set of data-oriented augmentations and manipulations that TorchSig developers have commonly found to aid with the training of communications signal domain ML models. You might think of these analogously to the randomized augmentations in image rotation, noise, and local region manipulation that often enhance generalization and speed convergence during model training in other domains like computer vision. These are purely data processing transformations that are not meant to represent real world signal effects and conditions. 

As of TorchSig v2.1.0, the current ML Transform definition randomly chooses among a variety of sample dropping, I/Q channel swapping, slope addition and time series reversal augmentations.

## Impairments Level 0
Impairments **level 0** comprises a pristine, unimpaired TorchSig signal generation environment that only exists in digital simulations. No additional signal impairments are present, and only the Machine Learning Transforms are applied. This level can usefully serve as a TorchSig baseline useful for data pipeline validation or as a basis for adding your own custom list of Transforms when building your own environments.

In [ ]:
impairments = Impairments(level=0)
burst_impairments = impairments.signal_transforms
whole_signal_impairments = impairments.dataset_transforms

burst_impairments, whole_signal_impairments

## Impairments Level 1
Impairments **level 1** models an RF benchtop or 'wired' channel configuration where plausible Commercial Off The Shelf (COTS) transmitter and recevier hardware apply typical signal nonidealities, but the wired channel is a cable that ensures the path from transmitter to receiver is stable and benign.

Impairment level 1 introduces a list transmitter hardware Transforms applied to signals after modulation, but prior to traversing the wired channel. This order and the default Transform parameters model the sequence those linear and nonlinear effects would apply to each transmitted signal. 

As of TorchSig v2.0.0, the level 1 transmitter Transform list is:
            - RandomApply(Quantize(),0.75),
            - RandomApply(PassbandRipple(),0.75),
            - RandomApply(IQImbalance(),0.25),
            - RandomApply(CarrierPhaseNoise(),0.75),
            - RandomApply(CarrierFrequencyDrift(),0.75),
            - RandomApply(CarrierPhaseOffset(),1.0),
            - RandomApply(IntermodulationProducts(),0.5),
            - RandomApply(NonlinearAmplifier(),0.75),
            - RandomApply(Spurs(),0.75),
            - RandomApply(SpectralInversion(),0.25)

Similarly, level 1 introduces a list of receiver hardware Transforms applied to signals after the transmitter and propagation channels cause their effects. 

As of TorchSig v2.0.0, the level 1 receiver Transform list is:
            - RandomApply(IntermodulationProducts(),0.5),
            - RandomApply(NonlinearAmplifier(),0.75),
            - RandomApply(CoarseGainChange(),0.25),
            - RandomApply(Spurs(),0.75),
            - RandomApply(IQImbalance(),0.5),
            - RandomApply(CarrierPhaseNoise(),0.75),
            - RandomApply(CarrierFrequencyDrift(),0.75),
            - RandomApply(CarrierPhaseOffset(),1.0),
            - RandomApply(PassbandRipple(),0.75),
            - RandomApply(Quantize(),0.75),
            - RandomApply(DigitalAGC(),0.25),

The Machine Learning Transforms are applied after the receiver impairments.

In [ ]:
impairments = Impairments(level=1)
burst_impairments = impairments.signal_transforms
whole_signal_impairments = impairments.dataset_transforms

burst_impairments, whole_signal_impairments

## Impairments Level 2
Impairments **level 2** models an RF over-the-air environment by introducing a wireless channel model using the TorchSig Fading Transform. Therefore, level 2 has all the effects of level 1 (including ML Transforms), plus the randomized addition of a fading channel. As of TorchSig v2.0.0, by default this wireless channel is a straightfoward Rayleigh fading model.

In [ ]:
impairments = Impairments(level=2)
burst_impairments = impairments.signal_transforms
whole_signal_impairments = impairments.dataset_transforms

burst_impairments, whole_signal_impairments